# Evaluate a RAG application

This example uses [Langchain](https://www.langchain.com) and [Giskard](https://github.com/Giskard-AI/giskard) to evaluate the quality of a RAG application.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = "gpt-3.5-turbo"

## Scrape the Website and Split the Content

In [4]:
# from langchain_community.document_loaders import WebBaseLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)

# loader = WebBaseLoader("https://www.ml.school/")
# documents = loader.load_and_split(text_splitter)
# documents

[Document(metadata={'source': 'https://www.ml.school/', 'title': "Building Machine Learning Systems That Don't Suck", 'description': "A live, interactive program that'll help you build production-ready machine learning systems from the ground up.", 'language': 'en'}, page_content='Building Machine Learning Systems That Don\'t Suck"This is the best machine learning course I\'ve done. Worth every cent."Jose Reyes, AI/ML at Cevo AustraliaBuilding Machine Learning Systems That Don\'t SuckA live, interactive program that\'ll help you build production-ready machine learning systems from the ground up.Next cohort:\xa0February 3 - 20, 2025Check the schedule for more details about upcoming cohorts.I want to join!Sign inLearn how to design, build, deploy, and scale machine learning systems to solve real-world problems.I\'ll lose my mind if I see another book or course teaching people the same basic ideas for the hundredth time. Most people are stuck in beginner mode, and finding help to solve re

In [35]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)

# Specify the path to your PDF file
loader = PyPDFLoader('./2021-q3-alphabet-10q.pdf')

# Load and split the PDF document
documents = loader.load_and_split(text_splitter)

# Output the documents
documents

[Document(metadata={'source': './2021-q3-alphabet-10q.pdf', 'page': 0}, page_content='UNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\n________________________________________________________________________________________\nFORM 10-Q \n________________________________________________________________________________________\n(Mark One)\n☒ QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the quarterly period ended September 30, 2021\nOR\n☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the transition period from _______ to _______\nCommission file number: 001-37580 \n________________________________________________________________________________________\nAlphabet Inc. \n(Exact name of registrant as specified in its charter)\n________________________________________________________________________________________\nDelaware 61-1767919\n(State or other jurisdiction of incor

## Load the Content in a Vector Store

In [36]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    documents, embedding=OpenAIEmbeddings()
)

## Create a Knowledge Base

Let's start by loading the content in a pandas DataFrame.

In [37]:
import pandas as pd

df = pd.DataFrame([d.page_content for d in documents], columns=["text"])
df.head(10)

,text
0,UNITED STATES\nSECURITIES AND EXCHANGE COMMISS...
1,"Mountain View, CA 94043\n(Address of principal..."
2,Indicate by check mark whether the registrant ...
3,complying with any new or revised financial ac...
4,Alphabet Inc.\nForm 10-Q\nFor the Quarterly Pe...
5,Item 1A Risk Factors 48\nItem 2 Unregistered S...
6,Note About Forward-Looking Statements\nThis Qu...
7,• our expectation that the portion of our reve...
8,• our expectation that our foreign exchange ri...
9,"marketing expenses, and general and administra..."


We can now create a Knowledge Base using the DataFrame we created before.

In [38]:
from giskard.rag import KnowledgeBase

knowledge_base = KnowledgeBase(df)

## Generate the Test Set

In [40]:
from giskard.rag import generate_testset

testset = generate_testset(
    knowledge_base,
    num_questions=60,
    agent_description="A chatbot answering questions about the earnings of Alphabet Inc. in Q3, 2021",
)

Generating questions: 100%|██████████| 60/60 [07:50<00:00,  7.83s/it]


Let's display a few samples from the test set.

In [41]:
test_set_df = testset.to_pandas()

for index, row in enumerate(test_set_df.head(3).iterrows()):
    print(f"Question {index + 1}: {row[1]['question']}")
    print(f"Reference answer: {row[1]['reference_answer']}")
    print("Reference context:")
    print(row[1]['reference_context'])
    print("******************", end="\n\n")


Question 1: Where can I find Google's corporate governance information?
Reference answer: Google's corporate governance information, including certificate of incorporation, bylaws, governance guidelines, board committee charters, and code of conduct, is available on their investor relations website under the heading 'Other.'
Reference context:
Document 171: filings, investor events, press and earnings releases, and blogs. We also share Google news and product updates 
on Google’s Keyword blog at https://www.blog.google/, which may be of interest or material to our investors. 
Further, corporate governance information, including our certificate of incorporation, bylaws, governance guidelines, 
board committee charters, and code of conduct, is also available on our investor relations website under the 
heading "Other." The content of our websites are not incorporated by reference into this Quarterly Report on Form 
10-Q or in any other report or document we file with the SEC, and any ref

Let's now save the test set to a file:

In [43]:
testset.save("test-set-alphabet-q1-2021.jsonl")

## Prepare the Prompt Template

In [16]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: Here is some context

Question: Here is a question



## Create the RAG Chain

Create a retriever from the Vector Store that will allow us to get the top similar documents to a given question.

In [17]:
retriever = vectorstore.as_retriever()
retriever.get_relevant_documents("What is the Machine Learning School?")

C:\Users\agnib\AppData\Local\Temp\ipykernel_22380\814621628.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("What is the Machine Learning School?")


[Document(metadata={'source': 'https://www.ml.school/', 'title': "Building Machine Learning Systems That Don't Suck", 'description': "A live, interactive program that'll help you build production-ready machine learning systems from the ground up.", 'language': 'en'}, page_content="program will help you unlearn what you think machine learning is. It's a practical, hands-on class where you'll learn from years of experience and real-world examples.When you join, you get lifetime access to the following:18 hours of live, interactive sessions. We'll use this time to discuss the first principles behind building machine learning systems.10 hours of step-by-step coding instructions. These practical sessions will show you how to build an end-to-end system from scratch.A final project where you'll build a complete solution and receive direct feedback on your work.100 coding assignments and practice questions.The entire source code of a working production system. It's yours. You can change and us

We can now create our chain.

In [20]:
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | StrOutputParser()
)

C:\Users\agnib\AppData\Local\Temp\ipykernel_22380\2743684902.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)


Let's make sure the chain works by testing it with a simple question.

In [21]:
chain.invoke({"question": "What is the Machine Learning School?"})

'The Machine Learning School is a live, interactive program that helps individuals build production-ready machine learning systems from scratch. It offers practical, hands-on classes, coding instructions, a final project, coding assignments, access to a private community, direct access to instructors, and lifetime access to past and future cohorts.'

## Evaluating the Model on the Test Set

We need to create a function that invokes the chain with a specific question and returns the answer.

In [22]:
def answer_fn(question, history=None):
    return chain.invoke({"question": question})

We can now use the `evaluate()` function to evaluate the model on the test set. This function will compare the answers from the chain with the reference answers in the test set.

In [23]:
from giskard.rag import evaluate

report = evaluate(answer_fn, testset=testset, knowledge_base=knowledge_base)

CorrectnessMetric evaluation: 100%|██████████| 60/60 [02:16<00:00,  2.27s/it]


Let now display the report.

Here are the five components of our RAG application:

* **Generator**: This is the LLM used in the chain to generate the answers.
* **Retriever**: This is the retriever that fetches relevant documents from the knowledge base according to a query.
* **Rewriter**: This is a component that rewrites the user query to make it more relevant to the knowledge base or to account for chat history.
* **Router**: This is a component that filters the query of the user based on his intentions.
* **Knowledge Base**: This is the set of documents given to the RAG to generate the answers.

In [24]:
display(report)

Loading BokehJS ...

In [25]:
report.to_html("report.html")

We can display the correctness results organized by question type.

In [26]:
report.correctness_by_question_type()

,correctness
question_type,
complex,1.0
conversational,0.4
distracting element,0.4
double,0.8
simple,0.7
situational,0.6


We can also display the specific failures.

In [27]:
report.get_failures()

,question,reference_answer,reference_context,conversation_history,metadata,agent_answer,correctness,correctness_reason
id,,,,,,,,
21b81dea-8d65-4b4c-a501-e7e28fabce54,What are some of the topics covered in the sec...,The second session of the course covers topics...,Document 7: to determine how much data you nee...,[],"{'question_type': 'simple', 'seed_document_id'...",The topics covered in the second session of th...,False,The agent missed mentioning the zero-rule algo...
70e621dd-1903-43ef-9737-bce15a9449e9,What skills will I gain from this Machine Lear...,You'll come out with practical skills and insi...,Document 3: that make systems work.You are rea...,[],"{'question_type': 'simple', 'seed_document_id'...",You'll come out with practical skills and insi...,False,"The agent provided a very general answer, but ..."
6febe0b2-71a4-4f54-b12b-b5c910e9427e,What are some of the topics covered in the sec...,The second session of the course covers topics...,Document 7: to determine how much data you nee...,[],"{'question_type': 'simple', 'seed_document_id'...",The topics covered in the second session of th...,False,The agent missed mentioning 'normalization and...
96922868-8c9c-4799-97d3-bb0f0a29df3b,Considering the course content covered until 2...,"Session 2 covers topics such as data cleaning,...",Document 7: to determine how much data you nee...,[],"{'question_type': 'distracting element', 'seed...",Session 2 covered topics such as data cleaning...,False,The agent missed mentioning the zero-rule algo...
321c31d4-e195-4dae-92d3-0f8c75cc07d6,"What practical skills and insights, relevant t...",You'll gain practical skills and insights into...,Document 3: that make systems work.You are rea...,[],"{'question_type': 'distracting element', 'seed...",You'll come out with practical skills and insi...,False,The agent's answer was partially correct but i...
340a2da0-deba-416e-bb71-40227c528bd2,In the context of the Machine Learning School ...,The instructor of the Machine Learning School ...,"Document 9: learning, beginners will find the ...",[],"{'question_type': 'distracting element', 'seed...",Juan Olano,False,The agent stated that the instructor is Juan O...
ed4c5c42-9abc-4afd-b8c9-b16c8d04d06d,Who possesses the copyright for the content of...,The copyright for the content is held by Tidei...,"Document 10: then, thousands of students have ...",[],"{'question_type': 'distracting element', 'seed...",The copyright for the content of the machine l...,False,The agent stated that the copyright belongs to...
c0e9ccfb-3f6d-452b-946f-c09e79903825,What are the benefits of joining the machine l...,"When you join the machine learning program, yo...",Document 6: as you'd like. No restrictions.Enj...,[],"{'question_type': 'distracting element', 'seed...",The benefits of joining the machine learning p...,False,The agent's answer focused on the technical as...
5e31f866-28da-4e68-9b36-ae9e44369efe,Considering that beginners might find the pace...,"When you join the machine learning program, yo...",Document 1: program will help you unlearn what...,[],"{'question_type': 'distracting element', 'seed...",The benefits of joining the machine learning p...,False,"The agent's answer was partially correct, but ..."


## Creating a Test Suite

We can create a test suite and use it to compare different models.

Load the test set from disk.

In [28]:
from giskard.rag import QATestset

testset = QATestset.load("test-set.jsonl")

Create a Test Suite from the test set.

In [29]:
test_suite = testset.to_test_suite("Machine Learning School Test Suite")

We need a function that takes a DataFrame of questions, invokes the chain with each question, and returns the answers.

In [30]:
import giskard


def batch_prediction_fn(df: pd.DataFrame):
    return chain.batch([{"question": q} for q in df["question"].values])

We can now create a Giskard Model object to run our test suite.

In [31]:
giskard_model = giskard.Model(
    model=batch_prediction_fn,
    model_type="text_generation",
    name="Machine Learning School Question and Answer Model",
    description="This model answers questions about the Machine Learning School website.",
    feature_names=["question"], 
)

2024-11-07 18:58:02,726 pid:22380 MainThread giskard.models.automodel INFO     Your 'prediction_function' is successfully wrapped by Giskard's 'PredictionFunctionModel' wrapper class.


Let's now run the test suite using the model we created before.

In [32]:
test_suite_results = test_suite.run(model=giskard_model)

2024-11-07 18:58:04,678 pid:22380 MainThread giskard.datasets.base INFO     Casting dataframe columns from {'question': 'object'} to {'question': 'object'}
2024-11-07 18:58:12,516 pid:22380 MainThread giskard.utils.logging_utils INFO     Predicted dataset with shape (60, 5) executed in 0:00:07.860728
2024-11-07 18:59:52,765 pid:22380 MainThread root         ERROR    An error happened during test execution for test: TestsetCorrectnessTest
Traceback (most recent call last):
  File "a:\github-profiles\turing\RAG\llm\venv\Lib\site-packages\giskard\core\suite.py", line 522, in run
    result = test_partial.giskard_test(**test_params).execute()
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "a:\github-profiles\turing\RAG\llm\venv\Lib\site-packages\giskard\registry\giskard_test.py", line 195, in execute
    return configured_validate_arguments(self.test_fn)(*self.args, **self.kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  Fi

We can display the results.

In [33]:
display(test_suite_results)

<TestSuiteResult (failed)>

## Integrating with Pytest

In [27]:
import ipytest

We can now integrate our test suite with Pytest.

In [36]:
%%ipytest

import pytest
from giskard.rag import QATestset
from giskard.testing.tests.llm import test_llm_correctness


@pytest.fixture
def dataset():
    testset = QATestset.load("test-set.jsonl")
    return testset.to_dataset()


@pytest.fixture
def model():
    return giskard_model


def test_chain(dataset, model):
    test_llm_correctness(model=model, dataset=dataset, threshold=0.5).assert_()

.2024-03-23 16:27:56,471 pid:46357 MainThread giskard.datasets.base INFO     Casting dataframe columns from {'question': 'object'} to {'question': 'object'}
2024-03-23 16:27:56,472 pid:46357 MainThread giskard.utils.logging_utils INFO     Predicted dataset with shape (60, 5) executed in 0:00:00.005269


.                                                                                           [100%]
========================================= warnings summary =========================================
../.venv/lib/python3.9/site-packages/_pytest/config/__init__.py:1276
  /Users/svpino/dev/llm/.venv/lib/python3.9/site-packages/_pytest/config/__init__.py:1276: PytestAssertRewriteWarning: Module already imported so cannot be rewritten: anyio
    self._mark_plugins_for_rewrite(hook)

t_66406511b9d84eb38baa6b0a22141dd0.py::test_llm_correctness
  /Users/svpino/dev/llm/.venv/lib/python3.9/site-packages/_pytest/python.py:199: PytestReturnNotNoneWarning: Expected None, but t_66406511b9d84eb38baa6b0a22141dd0.py::test_llm_correctness returned To execute the test call "execute()" method, which will be an error in a future version of pytest.  Did you mean to use `assert` instead of `return`?
    warnings.warn(

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
2 passed, 2 warni